<a href="https://colab.research.google.com/github/platypus2000jp/Colaboratory/blob/main/%E8%B6%85%E8%A7%A3%E5%83%8F%E6%8A%80%E8%A1%93%E6%AF%94%E8%BC%83%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: 画像をアップロードします。

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving IMG_0960.JPG to IMG_0960.JPG
User uploaded file "IMG_0960.JPG" with length 3094593 bytes


In [5]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。

from google.colab import files
import cv2
import numpy as np
from PIL import Image

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

  # 画像を読み込む
  img = cv2.imdecode(np.frombuffer(uploaded[fn], np.uint8), cv2.IMREAD_UNCHANGED)

  # 超解像処理をここに追加 (例として、単純な拡大処理を使用)
  # 実際の超解像技術を使用する場合は、適切なライブラリやモデルを導入する必要があります。
  upscaled_img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

  # 画像をPIL Imageオブジェクトに変換
  pil_image = Image.fromarray(cv2.cvtColor(upscaled_img, cv2.COLOR_BGR2RGB))

  # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
  new_filename = fn.split('.')[0] + "_upscaled.webp"

  # WebP形式で保存
  pil_image.save(new_filename, 'webp')

  # ダウンロード
  files.download(new_filename)

Saving IMG_0960.JPG to IMG_0960 (2).JPG
User uploaded file "IMG_0960 (2).JPG" with length 3094593 bytes


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はBSRGAN, ICCV2021を使用してください。

!pip install basicsr
!pip install gfpgan
!pip install facexlib
!pip install -r requirements.txt

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch
from gfpgan import GFPGANer

# モデルをダウンロード
model_url = 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/RealESRGAN_x4plus.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)


# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'

In [7]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はReal-ESRGANを使用してください。

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch


!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt


# モデルをダウンロード
model_url = 'https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/RealESRGAN_x4plus.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)


# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.5.0/RealESRGAN_x4plus.pth" to /content/RealESRGAN_x4plus.pth



HTTPError: HTTP Error 404: Not Found

In [8]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はSwinIR (ours)を使用してください。

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch

!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt


# モデルをダウンロード
model_url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)


# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Downloading: "https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth" to /content/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth



100%|██████████| 136M/136M [00:00<00:00, 211MB/s]
<ipython-input-8-fa2416608f83>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_pat

RuntimeError: Error(s) in loading state_dict for RRDBNet:
	Missing key(s) in state_dict: "body.0.rdb1.conv1.weight", "body.0.rdb1.conv1.bias", "body.0.rdb1.conv2.weight", "body.0.rdb1.conv2.bias", "body.0.rdb1.conv3.weight", "body.0.rdb1.conv3.bias", "body.0.rdb1.conv4.weight", "body.0.rdb1.conv4.bias", "body.0.rdb1.conv5.weight", "body.0.rdb1.conv5.bias", "body.0.rdb2.conv1.weight", "body.0.rdb2.conv1.bias", "body.0.rdb2.conv2.weight", "body.0.rdb2.conv2.bias", "body.0.rdb2.conv3.weight", "body.0.rdb2.conv3.bias", "body.0.rdb2.conv4.weight", "body.0.rdb2.conv4.bias", "body.0.rdb2.conv5.weight", "body.0.rdb2.conv5.bias", "body.0.rdb3.conv1.weight", "body.0.rdb3.conv1.bias", "body.0.rdb3.conv2.weight", "body.0.rdb3.conv2.bias", "body.0.rdb3.conv3.weight", "body.0.rdb3.conv3.bias", "body.0.rdb3.conv4.weight", "body.0.rdb3.conv4.bias", "body.0.rdb3.conv5.weight", "body.0.rdb3.conv5.bias", "body.1.rdb1.conv1.weight", "body.1.rdb1.conv1.bias", "body.1.rdb1.conv2.weight", "body.1.rdb1.conv2.

In [9]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はSwinIR-Large (ours)を使用してください。

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch

!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt


# モデルをダウンロード (SwinIR-Large)
model_url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)


# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


<ipython-input-9-cff8dda26216>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['params_ema'])


RuntimeError: Error(s) in loading state_dict for RRDBNet:
	Missing key(s) in state_dict: "body.0.rdb1.conv1.weight", "body.0.rdb1.conv1.bias", "body.0.rdb1.conv2.weight", "body.0.rdb1.conv2.bias", "body.0.rdb1.conv3.weight", "body.0.rdb1.conv3.bias", "body.0.rdb1.conv4.weight", "body.0.rdb1.conv4.bias", "body.0.rdb1.conv5.weight", "body.0.rdb1.conv5.bias", "body.0.rdb2.conv1.weight", "body.0.rdb2.conv1.bias", "body.0.rdb2.conv2.weight", "body.0.rdb2.conv2.bias", "body.0.rdb2.conv3.weight", "body.0.rdb2.conv3.bias", "body.0.rdb2.conv4.weight", "body.0.rdb2.conv4.bias", "body.0.rdb2.conv5.weight", "body.0.rdb2.conv5.bias", "body.0.rdb3.conv1.weight", "body.0.rdb3.conv1.bias", "body.0.rdb3.conv2.weight", "body.0.rdb3.conv2.bias", "body.0.rdb3.conv3.weight", "body.0.rdb3.conv3.bias", "body.0.rdb3.conv4.weight", "body.0.rdb3.conv4.bias", "body.0.rdb3.conv5.weight", "body.0.rdb3.conv5.bias", "body.1.rdb1.conv1.weight", "body.1.rdb1.conv1.bias", "body.1.rdb1.conv2.weight", "body.1.rdb1.conv2.

In [10]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はSRGANを使用してください。

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch

!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt

# モデルをダウンロード (SwinIR-Large)
model_url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)


# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


<ipython-input-10-385e525a77de>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['params_ema'])


RuntimeError: Error(s) in loading state_dict for RRDBNet:
	Missing key(s) in state_dict: "body.0.rdb1.conv1.weight", "body.0.rdb1.conv1.bias", "body.0.rdb1.conv2.weight", "body.0.rdb1.conv2.bias", "body.0.rdb1.conv3.weight", "body.0.rdb1.conv3.bias", "body.0.rdb1.conv4.weight", "body.0.rdb1.conv4.bias", "body.0.rdb1.conv5.weight", "body.0.rdb1.conv5.bias", "body.0.rdb2.conv1.weight", "body.0.rdb2.conv1.bias", "body.0.rdb2.conv2.weight", "body.0.rdb2.conv2.bias", "body.0.rdb2.conv3.weight", "body.0.rdb2.conv3.bias", "body.0.rdb2.conv4.weight", "body.0.rdb2.conv4.bias", "body.0.rdb2.conv5.weight", "body.0.rdb2.conv5.bias", "body.0.rdb3.conv1.weight", "body.0.rdb3.conv1.bias", "body.0.rdb3.conv2.weight", "body.0.rdb3.conv2.bias", "body.0.rdb3.conv3.weight", "body.0.rdb3.conv3.bias", "body.0.rdb3.conv4.weight", "body.0.rdb3.conv4.bias", "body.0.rdb3.conv5.weight", "body.0.rdb3.conv5.bias", "body.1.rdb1.conv1.weight", "body.1.rdb1.conv1.bias", "body.1.rdb1.conv2.weight", "body.1.rdb1.conv2.

In [11]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はSRCNNを使用してください。

!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch

# モデルをダウンロード (SwinIR-Large)
model_url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)


# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


<ipython-input-11-fabdef4da3ac>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['params_ema'])


RuntimeError: Error(s) in loading state_dict for RRDBNet:
	Missing key(s) in state_dict: "body.0.rdb1.conv1.weight", "body.0.rdb1.conv1.bias", "body.0.rdb1.conv2.weight", "body.0.rdb1.conv2.bias", "body.0.rdb1.conv3.weight", "body.0.rdb1.conv3.bias", "body.0.rdb1.conv4.weight", "body.0.rdb1.conv4.bias", "body.0.rdb1.conv5.weight", "body.0.rdb1.conv5.bias", "body.0.rdb2.conv1.weight", "body.0.rdb2.conv1.bias", "body.0.rdb2.conv2.weight", "body.0.rdb2.conv2.bias", "body.0.rdb2.conv3.weight", "body.0.rdb2.conv3.bias", "body.0.rdb2.conv4.weight", "body.0.rdb2.conv4.bias", "body.0.rdb2.conv5.weight", "body.0.rdb2.conv5.bias", "body.0.rdb3.conv1.weight", "body.0.rdb3.conv1.bias", "body.0.rdb3.conv2.weight", "body.0.rdb3.conv2.bias", "body.0.rdb3.conv3.weight", "body.0.rdb3.conv3.bias", "body.0.rdb3.conv4.weight", "body.0.rdb3.conv4.bias", "body.0.rdb3.conv5.weight", "body.0.rdb3.conv5.bias", "body.1.rdb1.conv1.weight", "body.1.rdb1.conv1.bias", "body.1.rdb1.conv2.weight", "body.1.rdb1.conv2.

In [12]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はDiffBirを使用してください。

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch

!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt

# モデルをダウンロード (SwinIR-Large)
model_url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)

# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


<ipython-input-12-f0557c4856fe>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['params_ema'])


RuntimeError: Error(s) in loading state_dict for RRDBNet:
	Missing key(s) in state_dict: "body.0.rdb1.conv1.weight", "body.0.rdb1.conv1.bias", "body.0.rdb1.conv2.weight", "body.0.rdb1.conv2.bias", "body.0.rdb1.conv3.weight", "body.0.rdb1.conv3.bias", "body.0.rdb1.conv4.weight", "body.0.rdb1.conv4.bias", "body.0.rdb1.conv5.weight", "body.0.rdb1.conv5.bias", "body.0.rdb2.conv1.weight", "body.0.rdb2.conv1.bias", "body.0.rdb2.conv2.weight", "body.0.rdb2.conv2.bias", "body.0.rdb2.conv3.weight", "body.0.rdb2.conv3.bias", "body.0.rdb2.conv4.weight", "body.0.rdb2.conv4.bias", "body.0.rdb2.conv5.weight", "body.0.rdb2.conv5.bias", "body.0.rdb3.conv1.weight", "body.0.rdb3.conv1.bias", "body.0.rdb3.conv2.weight", "body.0.rdb3.conv2.bias", "body.0.rdb3.conv3.weight", "body.0.rdb3.conv3.bias", "body.0.rdb3.conv4.weight", "body.0.rdb3.conv4.bias", "body.0.rdb3.conv5.weight", "body.0.rdb3.conv5.bias", "body.1.rdb1.conv1.weight", "body.1.rdb1.conv1.bias", "body.1.rdb1.conv2.weight", "body.1.rdb1.conv2.

In [13]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はFSRCNNを使用してください。

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch

!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt

# モデルをダウンロード (SwinIR-Large)
model_url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)

# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


<ipython-input-13-408addd04b1c>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['params_ema'])


RuntimeError: Error(s) in loading state_dict for RRDBNet:
	Missing key(s) in state_dict: "body.0.rdb1.conv1.weight", "body.0.rdb1.conv1.bias", "body.0.rdb1.conv2.weight", "body.0.rdb1.conv2.bias", "body.0.rdb1.conv3.weight", "body.0.rdb1.conv3.bias", "body.0.rdb1.conv4.weight", "body.0.rdb1.conv4.bias", "body.0.rdb1.conv5.weight", "body.0.rdb1.conv5.bias", "body.0.rdb2.conv1.weight", "body.0.rdb2.conv1.bias", "body.0.rdb2.conv2.weight", "body.0.rdb2.conv2.bias", "body.0.rdb2.conv3.weight", "body.0.rdb2.conv3.bias", "body.0.rdb2.conv4.weight", "body.0.rdb2.conv4.bias", "body.0.rdb2.conv5.weight", "body.0.rdb2.conv5.bias", "body.0.rdb3.conv1.weight", "body.0.rdb3.conv1.bias", "body.0.rdb3.conv2.weight", "body.0.rdb3.conv2.bias", "body.0.rdb3.conv3.weight", "body.0.rdb3.conv3.bias", "body.0.rdb3.conv4.weight", "body.0.rdb3.conv4.bias", "body.0.rdb3.conv5.weight", "body.0.rdb3.conv5.bias", "body.1.rdb1.conv1.weight", "body.1.rdb1.conv1.bias", "body.1.rdb1.conv2.weight", "body.1.rdb1.conv2.

In [14]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はESPCNを使用してください。

!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch

# モデルをダウンロード (SwinIR-Large)
model_url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)

# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


<ipython-input-14-a323b9fe7d77>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['params_ema'])


RuntimeError: Error(s) in loading state_dict for RRDBNet:
	Missing key(s) in state_dict: "body.0.rdb1.conv1.weight", "body.0.rdb1.conv1.bias", "body.0.rdb1.conv2.weight", "body.0.rdb1.conv2.bias", "body.0.rdb1.conv3.weight", "body.0.rdb1.conv3.bias", "body.0.rdb1.conv4.weight", "body.0.rdb1.conv4.bias", "body.0.rdb1.conv5.weight", "body.0.rdb1.conv5.bias", "body.0.rdb2.conv1.weight", "body.0.rdb2.conv1.bias", "body.0.rdb2.conv2.weight", "body.0.rdb2.conv2.bias", "body.0.rdb2.conv3.weight", "body.0.rdb2.conv3.bias", "body.0.rdb2.conv4.weight", "body.0.rdb2.conv4.bias", "body.0.rdb2.conv5.weight", "body.0.rdb2.conv5.bias", "body.0.rdb3.conv1.weight", "body.0.rdb3.conv1.bias", "body.0.rdb3.conv2.weight", "body.0.rdb3.conv2.bias", "body.0.rdb3.conv3.weight", "body.0.rdb3.conv3.bias", "body.0.rdb3.conv4.weight", "body.0.rdb3.conv4.bias", "body.0.rdb3.conv5.weight", "body.0.rdb3.conv5.bias", "body.1.rdb1.conv1.weight", "body.1.rdb1.conv1.bias", "body.1.rdb1.conv2.weight", "body.1.rdb1.conv2.

In [15]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はVDSRを使用してください。

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch

!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt

# モデルをダウンロード (SwinIR-Large)
model_url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)

# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


<ipython-input-15-65d11bd5157a>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['params_ema'])


RuntimeError: Error(s) in loading state_dict for RRDBNet:
	Missing key(s) in state_dict: "body.0.rdb1.conv1.weight", "body.0.rdb1.conv1.bias", "body.0.rdb1.conv2.weight", "body.0.rdb1.conv2.bias", "body.0.rdb1.conv3.weight", "body.0.rdb1.conv3.bias", "body.0.rdb1.conv4.weight", "body.0.rdb1.conv4.bias", "body.0.rdb1.conv5.weight", "body.0.rdb1.conv5.bias", "body.0.rdb2.conv1.weight", "body.0.rdb2.conv1.bias", "body.0.rdb2.conv2.weight", "body.0.rdb2.conv2.bias", "body.0.rdb2.conv3.weight", "body.0.rdb2.conv3.bias", "body.0.rdb2.conv4.weight", "body.0.rdb2.conv4.bias", "body.0.rdb2.conv5.weight", "body.0.rdb2.conv5.bias", "body.0.rdb3.conv1.weight", "body.0.rdb3.conv1.bias", "body.0.rdb3.conv2.weight", "body.0.rdb3.conv2.bias", "body.0.rdb3.conv3.weight", "body.0.rdb3.conv3.bias", "body.0.rdb3.conv4.weight", "body.0.rdb3.conv4.bias", "body.0.rdb3.conv5.weight", "body.0.rdb3.conv5.bias", "body.1.rdb1.conv1.weight", "body.1.rdb1.conv1.bias", "body.1.rdb1.conv2.weight", "body.1.rdb1.conv2.

In [16]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はDRCNを使用してください。

!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch

# モデルをダウンロード (SwinIR-Large)
model_url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)

# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


<ipython-input-16-17a36d0cef01>:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['params_ema'])


RuntimeError: Error(s) in loading state_dict for RRDBNet:
	Missing key(s) in state_dict: "body.0.rdb1.conv1.weight", "body.0.rdb1.conv1.bias", "body.0.rdb1.conv2.weight", "body.0.rdb1.conv2.bias", "body.0.rdb1.conv3.weight", "body.0.rdb1.conv3.bias", "body.0.rdb1.conv4.weight", "body.0.rdb1.conv4.bias", "body.0.rdb1.conv5.weight", "body.0.rdb1.conv5.bias", "body.0.rdb2.conv1.weight", "body.0.rdb2.conv1.bias", "body.0.rdb2.conv2.weight", "body.0.rdb2.conv2.bias", "body.0.rdb2.conv3.weight", "body.0.rdb2.conv3.bias", "body.0.rdb2.conv4.weight", "body.0.rdb2.conv4.bias", "body.0.rdb2.conv5.weight", "body.0.rdb2.conv5.bias", "body.0.rdb3.conv1.weight", "body.0.rdb3.conv1.bias", "body.0.rdb3.conv2.weight", "body.0.rdb3.conv2.bias", "body.0.rdb3.conv3.weight", "body.0.rdb3.conv3.bias", "body.0.rdb3.conv4.weight", "body.0.rdb3.conv4.bias", "body.0.rdb3.conv5.weight", "body.0.rdb3.conv5.bias", "body.1.rdb1.conv1.weight", "body.1.rdb1.conv1.bias", "body.1.rdb1.conv2.weight", "body.1.rdb1.conv2.

In [17]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はRED_Netを使用してください。

!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch


# モデルをダウンロード (SwinIR-Large)
model_url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)

# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


<ipython-input-17-76d2d597e23b>:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['params_ema'])


RuntimeError: Error(s) in loading state_dict for RRDBNet:
	Missing key(s) in state_dict: "body.0.rdb1.conv1.weight", "body.0.rdb1.conv1.bias", "body.0.rdb1.conv2.weight", "body.0.rdb1.conv2.bias", "body.0.rdb1.conv3.weight", "body.0.rdb1.conv3.bias", "body.0.rdb1.conv4.weight", "body.0.rdb1.conv4.bias", "body.0.rdb1.conv5.weight", "body.0.rdb1.conv5.bias", "body.0.rdb2.conv1.weight", "body.0.rdb2.conv1.bias", "body.0.rdb2.conv2.weight", "body.0.rdb2.conv2.bias", "body.0.rdb2.conv3.weight", "body.0.rdb2.conv3.bias", "body.0.rdb2.conv4.weight", "body.0.rdb2.conv4.bias", "body.0.rdb2.conv5.weight", "body.0.rdb2.conv5.bias", "body.0.rdb3.conv1.weight", "body.0.rdb3.conv1.bias", "body.0.rdb3.conv2.weight", "body.0.rdb3.conv2.bias", "body.0.rdb3.conv3.weight", "body.0.rdb3.conv3.bias", "body.0.rdb3.conv4.weight", "body.0.rdb3.conv4.bias", "body.0.rdb3.conv5.weight", "body.0.rdb3.conv5.bias", "body.1.rdb1.conv1.weight", "body.1.rdb1.conv1.bias", "body.1.rdb1.conv2.weight", "body.1.rdb1.conv2.

In [18]:
# prompt: アップロードした画像に超解像技術を適用し、適用後の画像に名前を付けてダウンロードして保存します。拡張子はWebPとします。超解像技術はDRRNを使用してください。

from google.colab import files
import cv2
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
import torch


!pip install basicsr
!pip install facexlib
!pip install -r requirements.txt


# モデルをダウンロード (SwinIR-Large)
model_url = 'https://github.com/JingyunLiang/SwinIR/releases/download/v0.0/003_realSR_BSRGAN_DFOWMFC_s64w8_SwinIR-L_x4_GAN.pth'
model_path = load_file_from_url(url=model_url, model_dir='./', progress=True, file_name=None)

# モデルの初期化
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
model.load_state_dict(torch.load(model_path)['params_ema'])
model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


def upscale_image(img_path, output_path):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).clamp(0, 1).squeeze(0).permute(1, 2, 0).cpu().numpy()

    output = (output * 255.0).round().astype(np.uint8)
    output = cv2.cvtColor(output, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, output)


uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

    # 画像を一時ファイルに保存
    with open(fn, 'wb') as f:
        f.write(uploaded[fn])

    # ファイル名を生成 (元のファイル名に "_upscaled.webp" を追加)
    new_filename = fn.split('.')[0] + "_upscaled.webp"

    # 超解像処理を実行
    upscale_image(fn, new_filename)

    # ダウンロード
    files.download(new_filename)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


<ipython-input-18-3e72e2c2ea0c>:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path)['params_ema'])


RuntimeError: Error(s) in loading state_dict for RRDBNet:
	Missing key(s) in state_dict: "body.0.rdb1.conv1.weight", "body.0.rdb1.conv1.bias", "body.0.rdb1.conv2.weight", "body.0.rdb1.conv2.bias", "body.0.rdb1.conv3.weight", "body.0.rdb1.conv3.bias", "body.0.rdb1.conv4.weight", "body.0.rdb1.conv4.bias", "body.0.rdb1.conv5.weight", "body.0.rdb1.conv5.bias", "body.0.rdb2.conv1.weight", "body.0.rdb2.conv1.bias", "body.0.rdb2.conv2.weight", "body.0.rdb2.conv2.bias", "body.0.rdb2.conv3.weight", "body.0.rdb2.conv3.bias", "body.0.rdb2.conv4.weight", "body.0.rdb2.conv4.bias", "body.0.rdb2.conv5.weight", "body.0.rdb2.conv5.bias", "body.0.rdb3.conv1.weight", "body.0.rdb3.conv1.bias", "body.0.rdb3.conv2.weight", "body.0.rdb3.conv2.bias", "body.0.rdb3.conv3.weight", "body.0.rdb3.conv3.bias", "body.0.rdb3.conv4.weight", "body.0.rdb3.conv4.bias", "body.0.rdb3.conv5.weight", "body.0.rdb3.conv5.bias", "body.1.rdb1.conv1.weight", "body.1.rdb1.conv1.bias", "body.1.rdb1.conv2.weight", "body.1.rdb1.conv2.